In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression as LR #線形回帰モデル
import seaborn as sns
#sns.set(font="IPAexGothic",style="white")


In [ ]:
#上限表示数を拡張
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 100)

In [ ]:
#data check
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("submit_sample.csv", header=None)

#data 中身閲覧
# train data
train.head()

In [ ]:
#test data都市名変換
change_city_df = pd.read_csv('change_city.csv')
change_city_df

In [ ]:
test_ex = pd.merge(test,change_city_df,how='left',on='City')

In [ ]:
test_ex=test_ex.drop(columns='City')

In [ ]:
test_ex=test_ex.rename(columns={'n_city':'City'})

In [ ]:
df_pm_mean = pd.DataFrame(train.groupby(['City','month'])['pm25_mid'].mean())
df_pm_mean = df_pm_mean.reset_index()
df_pm_mean

In [ ]:
df_pm_mean=df_pm_mean.rename(columns={'pm25_mid':'pm25_mean'})

In [ ]:
#train/test concat
df_all = pd.concat([train,test_ex])

In [ ]:
#meanをconcat
df_all=pd.merge(df_all,df_pm_mean,how='left',on=['City','month'])
df_all

In [ ]:
df_all['pm25_mean'].isnull().sum()

In [ ]:
#特徴量追加
df_all['co_range'] = df_all['co_max']-df_all['co_min']
df_all['o3_range']  = df_all['o3_max']-df_all['o3_min']
df_all['so2_range'] = df_all['so2_max']-df_all['so2_min']
df_all['no2_range'] = df_all['no2_max']-df_all['no2_min']
df_all['temperature_range'] = df_all['temperature_max']-df_all['temperature_min']
df_all['humidity_range'] = df_all['humidity_max']-df_all['humidity_min']
df_all['pressure_range'] = df_all['pressure_max']-df_all['pressure_min']
df_all['ws_range'] = df_all['ws_max']-df_all['ws_min']
df_all['dew_range'] = df_all['dew_max']-df_all['dew_min']

In [ ]:
#特徴量追加
df_all['co_so2_no2_mid']=df_all['co_mid']+df_all['so2_mid']+df_all['no2_mid']
df_all['month_day'] = df_all['month']*100+df_all['day']
#移動平均(3日単位：都市単位に求める必要あり)
#df_all['co_so2_no2_mid_3rel'] = df_all['co_so2_no2_mid'].rolling(3).sum()
#df_all['co_so2_no2_mid_bf1'] = df_all['co_so2_no2_mid'].shift(1)

In [ ]:
city_list = df_all['City'].unique()

In [ ]:
len(city_list)

In [ ]:
df_all['co_so2_no2_mid_3rel']=0
df_all['co_so2_no2_mid_7rel']=0
#df_all['co_so2_no2_mid_10rel']=0
#df_all['co_mid_3rel']=0
#df_all['so2_mid_3rel']=0
#df_all['no2_mid_3rel']=0
#df_all['co_so2_no2_mid_bf1']=0

In [ ]:
for city in city_list:
    df_tmp = df_all[df_all['City']==city].copy()
    df_tmp['co_so2_no2_mid_3rel'] = df_tmp['co_so2_no2_mid'].rolling(3).sum()
    df_tmp['co_so2_no2_mid_7rel'] = df_tmp['co_so2_no2_mid'].rolling(7).sum()
    #df_tmp['co_so2_no2_mid_10rel'] = df_tmp['co_so2_no2_mid'].rolling(10).sum()
    #df_tmp['co_mid_3rel'] = df_tmp['co_mid'].rolling(3).sum()
    #df_tmp['so2_mid_3rel'] = df_tmp['so2_mid'].rolling(3).sum()
    #df_tmp['no2_mid_3rel'] = df_tmp['no2_mid'].rolling(3).sum()
    #df_tmp['co_so2_no2_mid_bf1'] = df_tmp['co_so2_no2_mid'].shift(1)
    df_all['co_so2_no2_mid_3rel'] = df_all['co_so2_no2_mid_3rel'].where(df_all['City']!=city,df_tmp['co_so2_no2_mid_3rel'])
    df_all['co_so2_no2_mid_7rel'] = df_all['co_so2_no2_mid_7rel'].where(df_all['City']!=city,df_tmp['co_so2_no2_mid_7rel'])
    #df_all['co_so2_no2_mid_10rel'] = df_all['co_so2_no2_mid_10rel'].where(df_all['City']!=city,df_tmp['co_so2_no2_mid_10rel'])
    #df_all['co_mid_3rel'] = df_all['co_mid_3rel'].where(df_all['City']!=city,df_tmp['co_mid_3rel'])
    #df_all['so2_mid_3rel'] = df_all['so2_mid_3rel'].where(df_all['City']!=city,df_tmp['so2_mid_3rel'])
    #df_all['no2_mid_3rel'] = df_all['no2_mid_3rel'].where(df_all['City']!=city,df_tmp['no2_mid_3rel'])
    #df_all['co_so2_no2_mid_bf1'] = df_all['co_so2_no2_mid_bf1'].where(df_all['City']!=city,df_tmp['co_so2_no2_mid_bf1'])

In [ ]:
print(df_all['co_so2_no2_mid_3rel'].isnull().sum())
print(df_all['co_so2_no2_mid_7rel'].isnull().sum())
#print(df_all['co_so2_no2_mid_10rel'].isnull().sum())
#print(df_all['co_mid_3rel'].isnull().sum())
#print(df_all['so2_mid_3rel'].isnull().sum())
#print(df_all['no2_mid_3rel'].isnull().sum())
#print(df_all['co_so2_no2_mid_bf1'].isnull().sum())

In [ ]:
def month_offset(x):
    if x == 1 or x == 8:
        offset = 4
    elif x == 2 or x == 9:    
        offset = 2
    elif x == 3 or x == 10:
        offset = 0
    elif x == 4 or x == 11:
        offset = 5
    elif x == 5 or x == 12:
        offset = 3
    elif x == 6:
        offset = 1
    else:
        offset = 6
    return offset
#df_all['month_offset'] = df_all['month'].apply(month_offset)
df_all['year_week'] = df_all['month_day']/7
df_all['year_week'] = df_all['year_week'].round().astype(int)

In [ ]:
df_tr = df_all[df_all['pm25_mid'].notnull()]
df_pm_mean2 = pd.DataFrame(df_tr.groupby(['Country','year_week'])['pm25_mid'].mean())
df_pm_mean2 = df_pm_mean2.reset_index()
df_pm_mean2

In [ ]:
df_pm_mean2=df_pm_mean2.rename(columns={'pm25_mid':'pm25_mean2'})

In [ ]:
#meanをconcat
df_all=pd.merge(df_all,df_pm_mean2,how='left',on=['Country','year_week'])
df_all

In [ ]:
df_all['pm25_mean2'].isnull().sum()

In [ ]:
#特徴量Drop
#df_all = df_all.drop(["pm25_mean2"], axis=1)

In [ ]:
#特徴量Drop
#df_all = df_all.drop(['co_cnt','o3_cnt','so2_cnt',"no2_cnt","temperature_cnt","humidity_cnt","pressure_cnt","ws_cnt","dew_cnt"], axis=1)
#df_all = df_all.drop(['co_var','o3_var','so2_var',"no2_var","temperature_var","humidity_var","pressure_var","ws_var","dew_var"], axis=1)
#df_all = df_all.drop(['co_min','o3_min','so2_min',"no2_min","temperature_min","humidity_min","pressure_min","ws_min","dew_min"], axis=1)

In [ ]:
#Country,Cityの処理(一旦dummie変数化する)
df_all = pd.get_dummies(df_all,columns=['Country','City'])

In [ ]:
#test/train 分離
train = df_all[df_all['pm25_mid'].notnull()]
test = df_all[df_all['pm25_mid'].isnull()]

In [ ]:
train['pm25_mid'].isnull().sum()

In [ ]:
test['pm25_mid'].isnull().sum()

In [ ]:
test = test.drop('pm25_mid',axis=1)

In [ ]:
df_test = test.drop('id',axis=1)

In [ ]:
df_train = train.drop(['pm25_mid','id'],axis=1)
df_target = train['pm25_mid']

In [ ]:
#学習データ作成
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

def get_train_data(df_train,df_target):
    X_train, X_test, y_train, y_test = train_test_split(df_train, df_target, test_size=0.1,random_state=1)
    return X_train, X_test, y_train, y_test


In [ ]:
X_train, X_test, y_train, y_test = get_train_data(df_train,df_target)

In [ ]:
#学習モデル定義
def lightGBM_trainer(X_train_cv,y_train_cv,X_test_cv,y_test_cv):
    # 学習用
    train_data = lgb.Dataset(X_train_cv, y_train_cv,
                            free_raw_data=False)
    # 検証用
    eval_data = lgb.Dataset(X_test_cv, y_test_cv, reference=train_data,
                           free_raw_data=False)
    my_params  = {
        'objective': 'regression',
        'max_bin':250,
        'learning_rate': 0.05, # 学習率
        'max_depth': -1, # 木の数 (負の値で無制限)
        'num_leaves': 11, # 枝葉の数
        'metric': ('mean_absolute_error', 'mean_squared_error', 'rmse', 'multi'),
        'drop_rate': 0.15,
        'verbose': 0
    }
    print(my_params)
    # 学習
    evaluation_results = {}                                     # 学習の経過を保存する箱
    model_lgb = lgb.train(my_params, 
                          train_data,
                          valid_names=['train', 'valid'],
                          valid_sets=[train_data,eval_data],
                          evals_result=evaluation_results,
                          num_boost_round=10000,
                          #early_stopping_rounds=100,
                          verbose_eval=50)
    return model_lgb

In [ ]:
#評価関数
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def eval_model_lgb(model,X_test,y_test):
    # テストデータで予測する
    y_pred = model.predict(X_test)
    #y_pred_max = np.argmax(y_pred)
 
    score = np.sqrt(mean_squared_error(y_test,y_pred))
    #score = mean_absolute_error(y_test,y_pred)
    # 学習が終わったモデルをリストに入れておく
    models_lgm.append(model)
    sqr_scores_lgm.append(score)
    print(score)
    return y_pred

In [ ]:
#cat boost
from catboost import CatBoostRegressor, FeaturesData, Pool

#学習モデル定義
def catboost_trainer(X_train_cv,y_train_cv,X_test_cv,y_test_cv):

    #cat boost
    train_pool = Pool(X_train_cv, y_train_cv)
    validate_pool = Pool(X_test_cv, y_test_cv)
    
    model_cat = CatBoostRegressor(iterations=10000,
                                  task_type="GPU",
                                  learning_rate=0.03, 
                                  depth=9, 
                                  eval_metric="RMSE", 
                                  early_stopping_rounds=1000,
                                  verbose=1000)
    model_cat.fit(train_pool)
    return model_cat

def eval_model_cat(model,X_test,y_test):
    # テストデータで予測する
    y_pred = model.predict(X_test)
 
    #squared err score
    score = np.sqrt(mean_squared_error(y_test,y_pred))
    models_cat.append(model)
    sqr_scores_cat.append(score)
    print(score)
    return y_pred

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor

#学習モデル定義
def rf_trainer(X_train_cv,y_train_cv,X_test_cv,y_test_cv):
    #RandomFOREST
    model_rf = RandomForestRegressor(random_state=1)
    model_rf.fit(X_train_cv,y_train_cv)
    return model_rf

def eval_model_rf(model_rf,X_test,y_test):
    y_pred = model_rf.predict(X_test)
    #squared err score
    score = mean_squared_error(y_test,y_pred)
    models_rf.append(model_rf)
    sqr_scores_rf.append(score)
    print(np.sqrt(score))
    return y_pred

In [ ]:
#学習
#light GBM
import lightgbm as lgb


# 学習データの数だけの数列（0行から最終行まで連番）
row_no_list = list(range(len(y_train)))

#KFold
kf = KFold(n_splits=3)

models_lgm = []
models_cat = []
models_rf = []
sqr_scores_lgm = []
sqr_scores_cat = []
sqr_scores_rf = []

#y_train_index list
for train_cv_no, eval_cv_no in kf.split(row_no_list, y_train):
    # ilocで取り出す行を指定
    X_train_cv = X_train.iloc[train_cv_no, :]
    y_train_cv = pd.Series(y_train).iloc[train_cv_no]
    X_eval_cv = X_train.iloc[eval_cv_no, :]
    y_eval_cv = pd.Series(y_train).iloc[eval_cv_no]

    #Light GBM
    model_lgm = lightGBM_trainer(X_train_cv,y_train_cv,X_eval_cv,y_eval_cv)
    eval_model_lgb(model_lgm,X_eval_cv,y_eval_cv)

    #catboost
    model_cat = catboost_trainer(X_train_cv,y_train_cv,X_eval_cv,y_eval_cv)
    eval_model_cat(model_cat,X_eval_cv,y_eval_cv)

    #RF
    #model_rf = rf_trainer(X_train_cv,y_train_cv,X_eval_cv,y_eval_cv)
    #eval_model_rf(model_rf,X_eval_cv,y_eval_cv)


In [ ]:
from statistics import mean
print(mean(sqr_scores_lgm))
print(max(sqr_scores_lgm))
print(min(sqr_scores_lgm))

In [ ]:
print(mean(sqr_scores_cat))
print(max(sqr_scores_cat))
print(min(sqr_scores_cat))

In [ ]:
#print(np.sqrt(mean(sqr_scores_rf)))
#print(np.sqrt(max(sqr_scores_rf)))
#print(np.sqrt(min(sqr_scores_rf)))

In [ ]:
#アンサンブル
lgm = 10
cat = 34
pred_mix_1 = ((models_lgm[0].predict(X_test)*lgm)+(models_cat[0].predict(X_test)*cat))/(lgm+cat)
pred_mix_2 = ((models_lgm[1].predict(X_test)*lgm)+(models_cat[1].predict(X_test)*cat))/(lgm+cat)
pred_mix_3 = ((models_lgm[2].predict(X_test)*lgm)+(models_cat[2].predict(X_test)*cat))/(lgm+cat)
score1 = np.sqrt(mean_squared_error(y_test,pred_mix_1))
score2 = np.sqrt(mean_squared_error(y_test,pred_mix_2))
score3 = np.sqrt(mean_squared_error(y_test,pred_mix_3))
print(score1)
print(score2)
print(score3)
print((score1+score2+score3)/3)

In [ ]:
import matplotlib
# 日本語対応
matplotlib.rc('font', family='BIZ UDGothic')
def result_graph(pred,y_test):
    # 1.グラフのサイズを指定し、グラフの大きさを正方形になるように設定する
    plt.figure(figsize=(5,5))

    # 2.横軸を実測値、縦軸を予測値として、散布図を描く
    plt.scatter(y_test,pred)

    min_value = min(y_test)
    max_value = max(y_test)

    # 4.最小値と最大値を使い、x軸およびy軸の値域を指定する
    plt.xlim([min_value,max_value])
    plt.ylim([min_value,max_value])

    # 5.対角線を描く
    plt.plot([min_value,max_value],[min_value,max_value])

    #x軸とy軸に名前を付ける
    plt.xlabel("実績値")
    plt.ylabel("予測値")

    # 可視化結果を表示する為に必要な関数
    plt.show()

In [ ]:
result_graph(pred_mix_1,y_test)

In [ ]:
result_graph(pred_mix_2,y_test)

In [ ]:
result_graph(pred_mix_3,y_test)

In [ ]:
#高速テスト版
model_lgm_ = lightGBM_trainer(X_train,y_train,X_test,y_test)
model_cat_ = catboost_trainer(X_train,y_train,X_test,y_test)
#model_rf_ = rf_trainer(X_train,y_train,X_test,y_test)

In [ ]:
pred_lgm_=eval_model_lgb(model_lgm_,X_test,y_test)
pred_cat_=eval_model_cat(model_cat_,X_test,y_test)
#pred_rf_=eval_model_rf(model_rf_,X_test,y_test)

In [ ]:
#アンサンブル

#pred_mix_ = ((models_lgm_.predict(X_test)*lgm)+(models_cat_.predict(X_test)*cat)+(models_rf_.predict(X_test)*rf))/(lgm+cat+rf)
pred_mix_ = ((models_lgm_.predict(X_test)*lgm)+(models_cat_.predict(X_test)*cat))/(lgm+cat)
score_ = np.sqrt(mean_squared_error(y_test,pred_mix_))
print(score_)

In [ ]:
# 全データを用いたTraining
#Light GBM
model_lgm = lightGBM_trainer(df_train,df_target,X_test,y_test)

eval_model_lgb(model_lgm,X_test,y_test)
#eval_model_LR(model_lr,X_eval_cv,y_eval_cv)


In [ ]:
sqr_scores_lgm

In [ ]:
# 特徴量の重要度をプロットする
from matplotlib import pyplot as plt
lgb.plot_importance(model_lgm, figsize=(20, 50))
plt.show()

In [ ]:
pred_lgm = model_lgm.predict(df_test)
#pred_lr = model_lr.predict(df_test)

In [ ]:
pred_lgm

In [ ]:
# 全データを用いたTraining
#catboost
model_cat = catboost_trainer(df_train,df_target,X_test,y_test)

In [ ]:
pred_cat = model_cat.predict(df_test)

In [ ]:
pred_cat

In [ ]:
for col in df_test.columns:
    print(col)

In [ ]:
sample

In [ ]:
# testデータに適応
#sample[1] = pred_lgm
sample[1] = (pred_lgm*lgm+pred_cat*cat)/(lgm+cat)
sample

In [ ]:
sample.to_csv("sample_submit_v19.csv", index=None, header=None)